<a href="https://colab.research.google.com/github/qwiksilva/cs224w-github-rec/blob/master/BigQueryGithubWQuery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [0]:
%load_ext google.colab.data_table
# %unload_ext google.colab.data_table

In [0]:
# Fetch the largest 20 projects by the number of prs they have.
# Save output in a variable `df`
%%bigquery --project cs224w-257119 df
SELECT prs.num_prs, prs.base_repo_id, projects.name, projects.deleted
FROM (
  SELECT COUNT(base_repo_id) as num_prs, base_repo_id
  FROM `ghtorrent-bq.ght_2018_04_01.pull_requests`
  GROUP BY base_repo_id
) prs
INNER JOIN `ghtorrent-bq.ght_2018_04_01.projects` projects ON prs.base_repo_id = projects.id
ORDER BY prs.num_prs DESC
LIMIT 20


In [4]:
df

,num_prs,base_repo_id,name,deleted
0,82846,20170979,everypolitician-data,False
1,69340,70604300,signcla-probe-repo,False
2,55760,789,liferay-portal,False
3,54805,33253535,sauron,False
4,40082,9570147,homebrew-cask,False
5,33504,6866209,homebrew,False
6,31212,202,gaia,False
7,30909,1803391,snoopys-sandbox,False
8,28820,3777,nixpkgs,False
9,28289,22091370,kubernetes,False


In [0]:
# Fetch the largest 20 projects by number of unqiue commenters
%%bigquery --project cs224w-257119 df
SELECT COUNT(comments.comment_id) as num_comments, COUNT(DISTINCT comments.user_id) as num_commenters, prs.base_repo_id
FROM `ghtorrent-bq.ght_2018_04_01.pull_request_comments` comments
INNER JOIN `ghtorrent-bq.ght_2018_04_01.pull_requests` prs ON prs.id = comments.pull_request_id
GROUP BY prs.base_repo_id
ORDER BY num_commenters DESC
LIMIT 20

In [0]:
#Pulling the comment data; I threw in everything I thought may be useful

%%bigquery --project cs224w-257119 pr_comments_spark
SELECT pr_comments.pull_request_id AS pr_id, 
pr.pullreq_id as github_pr_id,
pr_comments.user_id as commenter_id, 
head_commits.author_id as head_commit_author_id,
head_commits.committer_id as head_commit_committer_id, 
base_commits.author_id as base_commit_author_id,
base_commits.committer_id as base_commit_committer_id,
pr.head_repo_id,
pr.base_repo_id,
base_project.owner_id as base_repo_owner,
head_project.owner_id as head_repo_owner,
pr.head_commit_id,
pr.base_commit_id,
pr.pullreq_id,
pr_comments.created_at as comment_created_at,
pr_comments.position as comment_position,
pr_comments.comment_id,
pr_comments.body as comment_body
FROM 
`ghtorrent-bq.ght_2018_04_01.pull_request_comments` pr_comments
INNER JOIN `ghtorrent-bq.ght_2018_04_01.pull_requests` pr
ON pr_comments.pull_request_id = pr.id
INNER JOIN `ghtorrent-bq.ght_2018_04_01.commits` head_commits
ON pr.head_commit_id = head_commits.id
INNER JOIN `ghtorrent-bq.ght_2018_04_01.commits` base_commits
ON pr.base_commit_id = base_commits.id
INNER JOIN `ghtorrent-bq.ght_2018_04_01.projects` base_project
ON base_project.id = pr.base_repo_id
INNER JOIN `ghtorrent-bq.ght_2018_04_01.projects` head_project
ON head_project.id = pr.head_repo_id
#Spark
WHERE pr.base_repo_id = 8196280

The head committer and author tends to be the same. The base committer and author can vary.
I checked, and the head commit ids match the user who submitted the pull request.
I'm still not sure how exactly to interpertet the base_commit_author and base_committer_ids. 
For rereference, this is the pullrequest corresponding to row 0 (github_pr_id 18476)

https://github.com/apache/spark/pull/18476

The usernames of the ids are as follows:

head_commit_author_id / head_commit_committer_id: **sadikovi**

base_commit_author_id: **liufengdb**

base_commit_committer_id: **zsxwing**


In [6]:
pr_comments_spark

,pr_id,github_pr_id,commenter_id,head_commit_author_id,head_commit_committer_id,base_commit_author_id,base_commit_committer_id,head_repo_id,base_repo_id,base_repo_owner,head_repo_owner,head_commit_id,base_commit_id,pullreq_id,comment_created_at,comment_position,comment_id,comment_body
0,31859265,19854,231317,6186517,6186517,3394061,3394061,15912312,8196280,13369,6186517,851427448,851176969,19854,2017-11-30 18:00:48+00:00,17,154154076,This change isn't valid and isn't related
1,31859265,19854,40383,6186517,6186517,3394061,3394061,15912312,8196280,13369,6186517,851427448,851176969,19854,2017-12-01 04:33:54+00:00,17,154266377,ditto. I think it is better to remove commente...
2,31859265,19854,6186517,6186517,6186517,3394061,3394061,15912312,8196280,13369,6186517,851427448,851176969,19854,2017-12-06 08:16:22+00:00,17,155168946,@srowen ： have deleted changes about Platform....
3,31859265,19854,40383,6186517,6186517,3394061,3394061,15912312,8196280,13369,6186517,851427448,851176969,19854,2017-12-01 04:32:05+00:00,7,154266242,I think the direct reference should be https:/...
4,31859265,19854,40383,6186517,6186517,3394061,3394061,15912312,8196280,13369,6186517,851427448,851176969,19854,2017-12-01 04:33:03+00:00,7,154266308,And we should not leave commented code.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130629,23245670,17575,231317,8172967,8172967,28831,9760153,47401183,8196280,13369,8172967,665447478,665273982,17575,2017-04-10 14:31:24+00:00,106,110671100,"OK no problem, leave it. Just riffing while we..."
130630,23245670,17575,231317,8172967,8172967,28831,9760153,47401183,8196280,13369,8172967,665447478,665273982,17575,2017-04-10 13:59:38+00:00,64,110662125,or does `itemsets.foreach(set => uniqItems ++=...
130631,23245670,17575,231317,8172967,8172967,28831,9760153,47401183,8196280,13369,8172967,665447478,665273982,17575,2017-04-11 07:07:26+00:00,70,110827932,"Nit, but this should unindent one unit"
130632,23245670,17575,8172967,8172967,8172967,28831,9760153,47401183,8196280,13369,8172967,665447478,665273982,17575,2017-04-10 14:07:20+00:00,48,110664282,"Yep, it can. It even avoids a useless cast. I..."


These next cells are so that we don't have to acess BigQeury everytime we want to work with the data.

In [0]:
#Download to csv
from google.colab import files

with open('pr_comments_spark.csv', 'w') as f:
  pr_comments_spark.to_csv(f)

files.download('pr_comments_spark.csv')

I downloaded a copy of the csv and put in under data/pr_comments_spark.csv in out github repo. Pull the repo to get the data locally on your computer. Then, when you want to use it again, you can upload your local copy by running the cell below.

I also discovered that collabs are saved as jupyter notebooks in the github repo, so I imagine you could also run them locally and get the data just by calling something like 

```
df = pd.read_csv('data/pr_comments_spark.csv')
```



In [9]:
import pandas as pd

uploaded = files.upload()

csv_file = None
for fn in uploaded.keys():
  csv_file = uploaded[fn]

if csv_file:
  spark_data = pd.read_csv(csv_file)